# IMPORTS

In [1]:
import pandas as pd

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from datetime import timedelta


# FONCTIONS

### FONCTION --> forcer l'affichage complet des lignes + colonnes

In [56]:
def myTotalDisplay(position):
  if position == 'on':
    # Définir le nombre de lignes à afficher sur "None" pour afficher toutes les lignes
    pd.set_option('display.max_rows', None)

    # Pour également afficher toutes les colonnes
    pd.set_option('display.max_columns', None)

    # Afficher un message pour indiquer que l'affichage de toutes les lignes et les colonnes a été activé
    print("L'affichage de toutes les lignes et toutes les colonnes a été activé")

  elif position == 'off':
    # Réinitialiser l'affichage du nombre maximum de lignes à la valeur par défaut
    pd.reset_option('display.max_rows')

    #  l'affichage du nombre maximum de colonnes à la valeur par défaut
    pd.reset_option('display.max_columns')
    
    # Afficher un message pour indiquer que l'affichage des lignes et des colonnes a été réinitialisé aux paramètres de défaut
    print("L'affichage des lignes et des colonnes a été réinitialisé aux paramètres de défaut")

  elif position == '?':
    print("taper 'on' ou 'off' pour activer ou désactiver l'affichage complète des tableaux")
  else:
    print("Mauvais argument. Tapez 'on' ou 'off' pour activer ou désactiver l'affichage complète des tableaux")


### FUNCTION --> Summary of Stats of all columns in a specified DF

In [5]:
from IPython.display import display  # import "display" for a formatted output

def generate_summary_table(df):
    print("Type de contenu\nMissing values & Descriptive Stats (mean, median, min, max...):")
    # Create an empty DF to store information
    summary_df = pd.DataFrame()

    # Add a column "Column Name", with the names of each column in the DF as values
    summary_df['Column Name'] = df.columns

    # Add a column "Data Type", with the data type of each column of the original DF
    summary_df['Data Type'] = df.dtypes.values

    # Add a column "Missing Values", with the sum of all the missing values for each column of the original DF
    summary_df['Missing Values'] = df.isnull().sum().values

    # Add a column "% of Missing Values", with the percentage of missing values per column relative to the total number of rows in the DataFrame
    summary_df['%_of Missing Values'] = (df.isnull().sum() / len(df) * 100).values

    # Calculate Descriptive Stats and transpose them in order to be able to combine with other stats,
    # Use 'include='all' to include all columns even, those without numerical values
    stats = df.describe(include='all').T

    # Merge the DFs to combine all stats
    summary_df = summary_df.merge(stats, left_on='Column Name', right_index=True, how='left')

    # Display the summary tableau
    return summary_df

# Call "tableau_function" and display result
#summary_table = generate_summary_table(df)
#display(summary_table)  


### FUNCTION --> Function to: 
 - 1) list columns, 
 - 2) identify numeric vs. categorical columns, 
 - 3) detect outliers, 
 - 4) heatmap

In [ ]:
def numeric_vs_categorical(df):
    # Display list of columns
    print("\nList of column names:")
    print(df.columns.tolist())

    # Identify a display numeric vs. categorical columns
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    categorical_columns = df.select_dtypes(include=['object']).columns

    print(f"\nNumeric columns:\n{numeric_columns}")
    print(f"\nCategorical columns:\n{categorical_columns}")

    # Columns that can be used for one-hot encoding
    print("\nColumns that can be used for one-hot encoding (<10 unique values):")
    dummies_columns = df.columns[df.nunique() < 10]
    print(dummies_columns.tolist())


# List of unique values less than 50%  of total lines (text brute)
    print("\nList of Unique values for columns with less than 50% unique entries:")
    total_rows = len(df)

for col in df.columns:
        unique_count = df[col].nunique()
        if unique_count < total_rows * 0.5:
            unique_values = list(df[col].unique())
            print(f"{col} : {unique_values}")

    # Check if there are any numeric columns
        if numeric_columns.empty:
            print("\n!!! The fonction for this section was not applied because there are no numeric columns.")
            print("!!! For machine learning, check if certain columns can be factorized of converted into numeric columns.")
        else:
        # Check for outliers in the numeric columns
            def detect_outliers(col):
                q1 = col.quantile(0.25)
                q3 = col.quantile(0.75)
                iqr = q3 - q1
                lower_bound = q1 - 1.5 * iqr
                upper_bound = q3 + 1.5 * iqr
                outliers = col[(col < lower_bound) | (col > upper_bound)]
                return outliers

        print("\Detect outliers for numeric columns:")
        outliers_count = {col: len(detect_outliers(df[col])) for col in numeric_columns}
        total_rows = len(df)
    
        ### Create a Dataframe with the number of outliers for each column
        outliers_df = pd.DataFrame(outliers_count, index=['Number of values'])

        # Add another line with the percentage of outliers in the column
        outliers_df.loc['% of values'] = (outliers_df.loc['Number of values'] / total_rows) * 100

        # Display the Dataframe
        display(outliers_df)

        # Generate a heatmap (correlation matrix) for the numerical columns only
        if not numeric_columns.empty:
            corr_matrix = df[numeric_columns].corr()
            mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
            plt.figure(figsize=(12, 8))
            sns.heatmap(corr_matrix,
                        annot=True,
                        cmap='coolwarm',
                        fmt='.1f',
                        linewidths=.5,
                        linecolor='white',
                        mask=mask,
                        annot_kws={"size": 6},
                        cbar_kws={"shrink": .8}
                        )

            plt.title('Heatmap Correlation Matrix')
            plt.xticks(rotation=45, ha='right', fontsize=8)
            plt.yticks(rotation=0, fontsize=8)
            plt.show()

### FONCTION --> Outliers - Correlation Matrix Heatmap + Boxplot

In [ ]:
# Create a HEATMAP of missing values
plt.figure(figsize=(12, 8))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Heatmap of missing values')
plt.show()

# First check to make sure there are numeric columns before continuing
if not numeric_columns.empty:
    # Display the distribution of numeric columns
    num_numeric_cols = len(numeric_columns)  # Détermine the total number of numeric columns
    cols_per_row = 3  # Define the number of columns per line for diplay 
    num_rows = (num_numeric_cols + cols_per_row - 1) // cols_per_row  # Calculate the number of columns needed

    if num_numeric_cols > 0:  # Make sure there are numeric columns
        fig, axes = plt.subplots(num_rows, cols_per_row, figsize=(12, 3 * num_rows))  # Create a figure and sub-charts
        axes = axes.flatten()  # Aplatir les axes pour les itérer facilement, en cas de figure 1D

        # Afficher les distributions des colonnes numériques
        for i, col in enumerate(numeric_columns):
            sns.histplot(df[col].dropna(), kde=True, ax=axes[i])
            axes[i].set_title(f'Distribution de {col}')

        # Si le nombre de graphiques est inférieur au nombre de sous-graphiques, cacher les axes inutilisés
        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])
        plt.tight_layout()
        plt.show()

    # BOXPLOT des OUTLIERS
    if num_numeric_cols > 0:  # Assurer qu'il y a bien des colonnes numériques pour les boxplots
        fig, axes = plt.subplots(num_rows, cols_per_row, figsize=(12, 1.5 * num_rows))  # Créer une figure et des sous-graphiques
        axes = axes.flatten()  # Aplatir les axes pour les itérer facilement, en cas de figure 1D

        # Afficher les boxplots pour détecter visuellement les outliers
        for i, col in enumerate(numeric_columns):
            sns.boxplot(x=df[col], ax=axes[i])
            axes[i].set_title(f'Boxplot de {col}')

        # Si le nombre de graphiques est inférieur au nombre de sous-graphiques, cacher les axes inutilisés
        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])
        plt.tight_layout()
        plt.show()
else:
    print("\n!!! Aucune colonne numérique détectée, les graphiques ne peuvent pas être affichés.")

# Ajouter une section pour le pairplot
if not numeric_columns.empty:
    print("\nVisualisation par pairplot :")
    sns.pairplot(df[numeric_columns])
    plt.show()
else:
    print("\n!!! La visualisation par pairplot est ignorée en raison de l'absence de données numériques.")

# Formulation d'hypothèses basée sur les observations
print("\nFormulation d'hypothèses :")
for col in numeric_columns:
    corr_with_others = df[numeric_columns].corr()[col].drop(col)
    if corr_with_others.abs().mean() > 0.5:
        print(f"Il semble y avoir une forte corrélation entre {col} et les autres variables numériques (corrélation moyenne : {corr_with_others.abs().mean():.2f}).")
for col in categorical_columnss:
    print(f"Pour {col}, on pourrait explorer l'effet de ses différentes catégories sur les variables numériques.")

# Restaurer les valeurs d'origine des options d'affichage
#(DESACTIVER L'AFFICHAGE DE LA TOTALITE DES COLONNES ET LIGNES D'UN TABLEAU PANDAS)
if options_changed:
    pd.set_option('display.max_rows', original_max_rows)
    pd.set_option('display.max_columns', original_max_columns)


### FONCTION --> Prepare data for PCA

In [ ]:
def prepare_data_for_pca(df, list_col=None, val_unique=50):
  import pandas as pd
  from sklearn.preprocessing import LabelEncoder
  import inspect
#    """
#    Prépare les données d'un DataFrame pour une analyse PCA et renomme le DataFrame de sortie en ajoutant 'my_' au nom d'origine.
#
#    Paramètres :
#    df : DataFrame à traiter.
#    list_col : Liste des colonnes texte à traiter. Si None, la fonction analyse toutes les colonnes texte.
#    val_unique : Pourcentage de valeurs uniques en dessous duquel la transformation LabelEncoder() est appliquée (par défaut 50).
#
#    Retour :
#    - DataFrame traité avec les colonnes supplémentaires et les colonnes encodées.
#    """

  df = df.copy()  # On travaille sur une copie pour ne pas modifier l'original
  df['myIndex'] = range(1, len(df) + 1)  # Ajout de la colonne myIndex

  # Gestion des colonnes de type date
  for col in df.select_dtypes(include=['datetime', 'object']):  # On cherche les colonnes de type date
      try:
          df[col] = pd.to_datetime(df[col])  # Tente de convertir en datetime
          if df[col].dt.time.nunique() > 1:  # S'il y a des heures incluses dans la date
              df['myYear'] = df[col].dt.year
              df['myMonth'] = df[col].dt.month
              df['myDay'] = df[col].dt.day
              df['myHour'] = df[col].dt.hour
              df['myMin'] = df[col].dt.minute
              df['mySec'] = df[col].dt.second
          else:  # Si la date n'inclut pas l'heure
              df['myYear'] = df[col].dt.year
              df['myMonth'] = df[col].dt.month
              df['myDay'] = df[col].dt.day

          df.drop(columns=[col], inplace=True)  # On supprime la colonne date originale
      except (ValueError, TypeError):
          # Si la conversion en datetime échoue, on ignore cette colonne
          pass

  # Gestion des colonnes de texte
  text_cols = list_col if list_col else df.select_dtypes(include=['object']).columns.tolist()
  for col in text_cols:
      if df[col].nunique() / len(df) < val_unique / 100:  # Si le nombre de valeurs uniques est < à val_unique %
          le = LabelEncoder()
          df[col + '_encoded'] = le.fit_transform(df[col])
          df.drop(columns=[col], inplace=True)  # Supprime la colonne texte d'origine une fois encodée

  # Récupérer le nom de la variable d'origine
  frame = inspect.currentframe().f_back
  variable_names = {id(v): k for k, v in frame.f_globals.items()}.get(id(df), None)

  # Renommer le DataFrame avec le préfixe "my_" suivi du nom de la variable d'origine
  new_df_name = 'my_' + variable_names if variable_names else 'my_df'
  print(f"Votre nouveau DataFrame se nomme {new_df_name}")

  return df

# Exemple d'utilisation :
# Supposons que ton DataFrame s'appelle dataSbu
# df_prepared = prepare_data_for_pca(dataSbu)


### FONCTION --> PCA

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def myAcp(df, group_col=None, variance_threshold=0.7):
    """
    Effectue une analyse en composantes principales (ACP/PCA) sur un DataFrame, en choisissant automatiquement le nombre de composantes
    nécessaires pour expliquer un certain pourcentage de la variance (défini par variance_threshold).

    Paramètres :
    df : DataFrame contenant les données à analyser.
    group_col : (optionnel) Nom de la colonne pour identifier les groupes catégoriels dans les observations.
    variance_threshold : Pourcentage de variance à expliquer par les composantes principales (par défaut 0.7).

    Retourne :
    - Le modèle PCA ajusté
    - Les données projetées dans l'espace des composantes principales
    - Liste des variables principales par composante avec leur poids
    - Visualisation du cercle des corrélations et du graphique des observations
    """

    # 1. Centrer et réduire les données
    if group_col:
        if group_col not in df.columns:
            raise ValueError(f"La colonne '{group_col}' n'existe pas dans le DataFrame.")
        X = df.drop(columns=[group_col])
        labels = df[group_col]
    else:
        X = df
        labels = None

    # Standardisation des données
    X_scaled = StandardScaler().fit_transform(X)

    # 2. Instancier l'objet PCA avec le seuil de variance
    pca = PCA(n_components=variance_threshold)  # n_components est maintenant un pourcentage de variance
    pca.fit(X_scaled)
    X_pca = pca.transform(X_scaled)

    # Affichage de la dimension de X_pca
    print(f"Nombre de dimensions sélectionnées pour expliquer {variance_threshold*100}% de la variance : {X_pca.shape[1]}")

    # Variance expliquée
    explained_variance = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance)

    print("\nVariance expliquée par chaque composante principale :")
    for i, var in enumerate(explained_variance):
        print(f"Composante {i+1}: {var*100:.2f}%")

    print("\nVariance cumulée :")
    for i, var in enumerate(cumulative_variance):
        print(f"Composante {i+1}: {var*100:.2f}% cumulée")

    # 3. Visualisation de la variance expliquée
    plt.figure(figsize=(8, 5))
    plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.5, align='center', label='Variance individuelle')
    plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', label='Variance cumulée')
    plt.axhline(y=variance_threshold, color='r', linestyle='--', label=f'Seuil {variance_threshold*100}%')
    plt.xlabel('Composantes principales')
    plt.ylabel('Variance expliquée')
    plt.legend(loc='best')
    plt.title('Variance expliquée par les composantes principales')
    plt.show()

    # 4. Identification des variables principales par composante avec le poids correspondant
    components = pca.components_
    loadings = pd.DataFrame(components.T, columns=[f'PC{i+1}' for i in range(components.shape[0])], index=X.columns if isinstance(df, pd.DataFrame) else df.columns)

    print("\nVariables principales avec le poids dans chaque composante :")
    for i in range(X_pca.shape[1]):
        print(f"\nComposante principale {i+1}:")
        loadings_per_component = loadings[f'PC{i+1}'].abs().sort_values(ascending=False)
        for var_name, value in loadings_per_component.items():  # Utiliser .items() à la place de .iteritems()
            percentage_contribution = value * 100
            print(f"  {var_name}: {percentage_contribution:.2f}%")

    # 5. Visualisation des deux premières composantes principales (Graphique des observations)
    plt.figure(figsize=(8, 6))
    if labels is not None:
        unique_labels = labels.unique()
        for label in unique_labels:
            plt.scatter(X_pca[labels == label, 0], X_pca[labels == label, 1], alpha=0.5, label=label)
        plt.legend(loc='best')
    else:
        plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5)

    plt.title('Projection des données sur les deux premières composantes principales (Observations)')
    plt.xlabel('Composante principale 1')
    plt.ylabel('Composante principale 2')
    plt.grid(True)
    plt.show()

    # 6. Cercle des corrélations
    fig, ax = plt.subplots(figsize=(8, 8))

    # Cercle unitaire
    circle = plt.Circle((0, 0), 1, color='black', fill=False, linestyle='--')
    ax.add_artist(circle)

    # Calculer les limites maximales de l'échelle en fonction des composantes
    max_val = np.abs(components[:2, :]).max()
    scale = 1.1 * max_val  # Échelle légèrement supérieure aux valeurs maximales

    for i, var in enumerate(X.columns if isinstance(df, pd.DataFrame) else df.columns):
        plt.arrow(0, 0, components[0, i], components[1, i], color='r', alpha=0.5, head_width=0.05)
        plt.text(components[0, i]*1.15, components[1, i]*1.15, var, fontsize=12)  # Ajuster la position du texte

    plt.xlim([-scale, scale])
    plt.ylim([-scale, scale])
    plt.xlabel(f'Composante principale 1 ({explained_variance[0]*100:.1f}%)')
    plt.ylabel(f'Composante principale 2 ({explained_variance[1]*100:.1f}%)')
    plt.grid(True)
    plt.title('Cercle des corrélations (Projection des variables)')
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.show()

    # Retourne le modèle PCA et les données projetées
    return pca, X_pca

# Exemple d'utilisation :
# df_prepared = pd.DataFrame(...)  # Vos données sous forme de DataFrame
# pca_model, X_pca = myAcp(df_prepared, variance_threshold=0.7)


### FONCTION --> Statistiques Descriptives for a specified column in a DF

In [ ]:
# Une fonction "calculate quartiles and outlier information"
# Fonction de calcul équivalent Describe 'light' : pour UNE SEULE COLONNE choisie dans un DF, avec affichage infos
 # Il faut définir le DF et la colonne 

def cal_quartiles__outlier_info(df,col):
  # Calculate the first quartile (Q1), second quartile (or median = Q2) and third quartiles (Q3) of the specified column
  q1 = df[col].quantile(0.25)
  q2_mediane = df[col].quantile(0.50)
  q3 = df[col].quantile(0.75)

  # Calculate the first quartile (Q1), second quartile (Q2) and third quartiles (Q3) of the specified column

# Calculate the interquartile range (IQR) as the difference between Q3 and Q1
  interquartile_range = q3 - q1
  
  # Define the minimum value for outliers based on the IQR
  outlier_min = q1 - (1.5 * interquartile_range)
  
  # Define the maximum value for outliers based on the IQR
  outlier_max = q3 + (1.5 * interquartile_range)

  # Find the maximum value in the column (for reference)
  r_max_jordan = df[col].max()

    # Count the number of values above the outlier maximum threshold
  line_outliers_max = (df[col] > outlier_max).sum()

    # Count the number of values below the outlier minimum threshold
  line_outliers_min = (df[col] < outlier_min).sum()

  print("Nombre de ligne et colonne:", df.shape)
  print("Quartile Q1 :", q1)
  print("Quartile Q2 (autrement dit, la MEDIANE) :", q2_mediane)
  print("Quartile Q3 :", q3)
  print("Outlier Min Threshold :", outlier_min)
  print("Outlier Max Threshold :", outlier_max)

    # Print the maximum value in the column
  print("Valeur Max de la colonne:", r_max_jordan)

    # Print the number of rows exceeding the outlier maximum threshold
  print("Nombre de lignes au-dessus du seuil MAX des OUTLIERS :", line_outliers_max)
  
    # Print the number of rows below the outlier minimum threshold
  print("Nombre de lignes en-dessous du seuil MIN des OUTLIERS :", line_outliers_min)

  return([q1,q2_mediane,q3,outlier_min,outlier_max, line_outliers_min, line_outliers_max, r_max_jordan])

### FONCTION --> Check for duplicates in a column

In [ ]:
# Check for duplicates in a DF column
def is_duplicate(df, col):
    # check if there are duplicates in the column, store info in a variable 'has_duplicates'
    has_duplicates = df[col].duplicated().any()

    # Count the number of duplicates
    count_duplicates = df[col].duplicated().sum()

    if has_duplicates:
        print(f"There are {count_duplicates} in the column '{col}' of the DF.")
    else:
        print(f"There are no duplicates in the column '{col}' of the DF.")

In [ ]:
# Colonnes qui peuvent être utilisées pour get_dummies
    print("\nListe des colonnes susceptibles d'être basculées en get_dummies (<10 valeurs uniques) :")
    dummies_columns = df.columns[df.nunique() < 10]
    print(dummies_columns.tolist())

    # Identifier les colonnes numériques et catégorielles
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=['object']).columns

## FONCTION --> Forcer affichage complet des colonnes 

"IATA_CODE" refers to the IATA Designator Code -->  used to identify an airline for commercial purposes in reservations, timetables, tickets, tariffs, air waybills and in telecommunications

# DF Airlines

In [6]:
df_airlines = pd.read_csv("/Users/kimberlyblack/Documents/POLE EMPLOI/DATA ANALYST/WCS/Projet 3/airlines.csv")

#previsualisation
df_airlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [7]:
df_airlines.columns

Index(['IATA_CODE', 'AIRLINE'], dtype='object')

In [3]:
df_airlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   IATA_CODE  14 non-null     object
 1   AIRLINE    14 non-null     object
dtypes: object(2)
memory usage: 356.0+ bytes


## Rename 'IATA_CODE' column in airline to 'AIRLINE_CODE'

In [5]:
df_airlines.replace('IATA_CODE','AIRLINE_CODE', inplace=True)

# DF Airports

In [7]:
df_airports = pd.read_csv("/Users/kimberlyblack/Documents/POLE EMPLOI/DATA ANALYST/WCS/Projet 3/airports.csv")

#previsualisation
df_airports.sample(30)

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
298,SUX,Sioux Gateway Airport,Sioux City,IA,USA,42.40260,-96.38437
10,ADQ,Kodiak Airport,Kodiak,AK,USA,57.74997,-152.49386
44,BRO,Brownsville/South Padre Island International A...,Brownsville,TX,USA,25.90683,-97.42586
70,CNY,Canyonlands Field,Moab,UT,USA,38.75496,-109.75484
154,ILG,Wilmington Airport,Wilmington,DE,USA,39.67872,-75.60653
16,AMA,Rick Husband Amarillo International Airport,Amarillo,TX,USA,35.21937,-101.70593
215,MSO,Missoula International Airport,Missoula,MT,USA,46.91631,-114.09056
237,PHF,Newport News/Williamsburg International Airport,Newport News,VA,USA,37.13190,-76.49299
214,MSN,Dane County Regional Airport,Madison,WI,USA,43.13986,-89.33751
188,LSE,La Crosse Regional Airport,La Crosse,WI,USA,43.87938,-91.25654


### Using summary stats of all columns function on df_airports

In [8]:
# Call "tableau_function" and display result
summary_table = generate_summary_table(df_airports)
display(summary_table)  


Type de contenu
Missing values & Descriptive Stats (mean, median, min, max...):


,Column Name,Data Type,Missing Values,%_of Missing Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
0,IATA_CODE,object,0,0.000000,322,322,ABE,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AIRPORT,object,0,0.000000,322,322,Lehigh Valley International Airport,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CITY,object,0,0.000000,322,308,Jackson,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,STATE,object,0,0.000000,322,54,TX,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,COUNTRY,object,0,0.000000,322,1,USA,322,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,LATITUDE,float64,3,0.931677,319.0,NaN,NaN,NaN,38.981244,8.616736,13.48345,33.65204,39.29761,43.154675,71.28545
6,LONGITUDE,float64,3,0.931677,319.0,NaN,NaN,NaN,-98.378964,21.523492,-176.64603,-110.839385,-93.40307,-82.722995,-64.79856


In [9]:
df_airports.columns

Index(['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')

In [5]:
df_airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IATA_CODE  322 non-null    object 
 1   AIRPORT    322 non-null    object 
 2   CITY       322 non-null    object 
 3   STATE      322 non-null    object 
 4   COUNTRY    322 non-null    object 
 5   LATITUDE   319 non-null    float64
 6   LONGITUDE  319 non-null    float64
dtypes: float64(2), object(5)
memory usage: 17.7+ KB


In [6]:
df_airports.isna().sum()

IATA_CODE    0
AIRPORT      0
CITY         0
STATE        0
COUNTRY      0
LATITUDE     3
LONGITUDE    3
dtype: int64

# DF Flights

In [2]:
df_flights = pd.read_csv("/Users/kimberlyblack/Documents/POLE EMPLOI/DATA ANALYST/WCS/Projet 3/flights.csv")

#previsualisation
df_flights.sample(20)




/var/folders/kl/2y01c7y51cj42bdww37cvc5c0000gn/T/ipykernel_15930/1035657769.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_flights = pd.read_csv("/Users/kimberlyblack/Documents/POLE EMPLOI/DATA ANALYST/WCS/Projet 3/flights.csv")


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
3011906,2015,7,8,3,WN,251,N526SW,AMA,DAL,1405,...,1650.0,100.0,0,0,NaN,1.0,0.0,4.0,95.0,0.0
681724,2015,2,15,7,NK,379,N623NK,ACY,MYR,630,...,829.0,28.0,0,0,NaN,10.0,0.0,0.0,0.0,18.0
4187776,2015,9,18,5,WN,493,N211WN,BHM,MDW,1015,...,1211.0,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4841424,2015,10,29,4,WN,172,N651SW,13495,11292,1855,...,2038.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1112439,2015,3,14,6,HA,156,N483HA,HNL,OGG,711,...,742.0,-5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5565352,2015,12,15,2,WN,2343,N463WN,HOU,MSY,1615,...,1705.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4690220,2015,10,20,2,DL,760,N627DL,10397,14893,940,...,1127.0,-18.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1996215,2015,5,7,4,AA,1134,N3JKAA,LGA,DFW,1621,...,1917.0,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3343429,2015,7,28,2,AA,1732,N750UW,CLT,DCA,735,...,839.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3503400,2015,8,6,4,WN,637,N778SW,PHX,STL,1400,...,1859.0,9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_flights.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

In [56]:
df_flights.shape

(5819079, 31)

In [71]:
df_flights.ORIGIN_AIRPORT.unique()

array(['ANC', 'LAX', 'SFO', 'SEA', 'LAS', 'DEN', 'SLC', 'PDX', 'FAI',
       'MSP', 'PHX', 'SJU', 'PBG', 'IAG', 'PSE', 'BQN', 'ORD', 'GEG',
       'HNL', 'ONT', 'MCO', 'BOS', 'HIB', 'ABR', 'MAF', 'DFW', 'MKE',
       'IAH', 'BNA', 'BRO', 'VPS', 'BOI', 'BJI', 'SGF', 'PHL', 'SBN',
       'RDD', 'EUG', 'IAD', 'BUF', 'PWM', 'JFK', 'CRP', 'PIA', 'FAT',
       'SMF', 'AUS', 'MCI', 'ATL', 'JAX', 'MFR', 'IDA', 'MSN', 'DCA',
       'SAT', 'CHS', 'SBA', 'SMX', 'IND', 'CLE', 'GSP', 'BDL', 'ABI',
       'RIC', 'BFL', 'OMA', 'RDM', 'FLL', 'CID', 'TPA', 'SYR', 'ROC',
       'TYR', 'LAN', 'XNA', 'GSO', 'EWR', 'PBI', 'RSW', 'OAK', 'PVD',
       'RNO', 'PIT', 'ABQ', 'MIA', 'BWI', 'LGA', 'TUL', 'LIT', 'MSY',
       'OKC', 'ATW', 'PNS', 'MEM', 'TYS', 'MHT', 'SAV', 'CLT', 'GRB',
       'ABE', 'JAN', 'OAJ', 'FAR', 'ERI', 'LEX', 'CWA', 'MSO', 'TTN',
       'AMA', 'CLL', 'HOU', 'JLN', 'MLI', 'RDU', 'CVG', 'MHK', 'MOB',
       'TLH', 'BHM', 'CAE', 'TXK', 'ACY', 'DTW', 'RAP', 'TUS', 'EAU',
       'DLH', 'FSD',

: 

## Delete CANCELLATION_REASON column

In [49]:
df_flights_clean = df_flights.drop('CANCELLATION_REASON', axis = 1)

## Replace NaN values in columns 'DEPARTURE_DELAY', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'

In [50]:
df_flights_clean = df_flights_clean.fillna({'DEPARTURE_DELAY':0, 'AIR_SYSTEM_DELAY':0, 'SECURITY_DELAY':0, 'AIRLINE_DELAY':0, 'LATE_AIRCRAFT_DELAY':0, 'WEATHER_DELAY':0})

df_flights_clean.head()


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,0.0,0.0,0.0,0.0,0.0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,0.0,0.0,0.0,0.0,0.0
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,0.0,0.0,0.0,0.0,0.0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,0.0,0.0,0.0,0.0,0.0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,0.0,0.0,0.0,0.0,0.0


In [51]:
df_flights_clean.isna().sum()

YEAR                        0
MONTH                       0
DAY                         0
DAY_OF_WEEK                 0
AIRLINE                     0
FLIGHT_NUMBER               0
TAIL_NUMBER             14721
ORIGIN_AIRPORT              0
DESTINATION_AIRPORT         0
SCHEDULED_DEPARTURE         0
DEPARTURE_TIME          86153
DEPARTURE_DELAY             0
TAXI_OUT                89047
WHEELS_OFF              89047
SCHEDULED_TIME              6
ELAPSED_TIME           105071
AIR_TIME               105071
DISTANCE                    0
WHEELS_ON               92513
TAXI_IN                 92513
SCHEDULED_ARRIVAL           0
ARRIVAL_TIME            92513
ARRIVAL_DELAY          105071
DIVERTED                    0
CANCELLED                   0
AIR_SYSTEM_DELAY            0
SECURITY_DELAY              0
AIRLINE_DELAY               0
LATE_AIRCRAFT_DELAY         0
WEATHER_DELAY               0
dtype: int64

In [55]:
df_flights_clean.shape

(5819079, 30)

In [52]:
df_flights_clean.to_csv('df_flights_clean.csv')

In [1]:
df_flights_clean.columns

NameError: name 'df_flights_clean' is not defined

### Observations about _delay columns

In [12]:
df_flights_reasons_exploration = df_flights[['DEPARTURE_DELAY', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']]

df_flights_reasons_exploration.sample(20)



,DEPARTURE_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
381076,-10.0,NaN,NaN,NaN,NaN,NaN
3436940,27.0,NaN,NaN,NaN,NaN,NaN
2914651,15.0,3.0,0.0,0.0,15.0,0.0
1186952,-6.0,NaN,NaN,NaN,NaN,NaN
2197288,-8.0,NaN,NaN,NaN,NaN,NaN
2168283,-7.0,32.0,0.0,0.0,0.0,0.0
1381715,1.0,NaN,NaN,NaN,NaN,NaN
188337,-3.0,NaN,NaN,NaN,NaN,NaN
3526838,-4.0,NaN,NaN,NaN,NaN,NaN
4665490,-6.0,NaN,NaN,NaN,NaN,NaN


In [31]:
df_flights_dd = df_flights_reasons_exploration.loc[df_flights_reasons_exploration['DEPARTURE_DELAY'] > 15]

df_flights_dd

,DEPARTURE_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
20,25.0,NaN,NaN,NaN,NaN,NaN
29,21.0,NaN,NaN,NaN,NaN,NaN
30,72.0,43.0,0.0,0.0,0.0,0.0
52,95.0,0.0,0.0,85.0,0.0,0.0
55,72.0,17.0,0.0,72.0,0.0,0.0
...,...,...,...,...,...,...
5819044,19.0,NaN,NaN,NaN,NaN,NaN
5819051,16.0,12.0,0.0,16.0,0.0,0.0
5819069,159.0,0.0,0.0,159.0,0.0,0.0
5819071,16.0,1.0,0.0,16.0,0.0,0.0


In [33]:
df_flights_dd_2 = df_flights_dd.loc[df_flights_reasons_exploration[['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']].isna().any(axis=1)]

df_flights_dd_2.value_counts('DEPARTURE_DELAY')

DEPARTURE_DELAY
16.0      24769
17.0      22382
18.0      19837
19.0      17451
20.0      15531
          ...  
382.0         1
381.0         1
380.0         1
379.0         1
1274.0        1
Name: count, Length: 422, dtype: int64

In [36]:
df_flights_arrivals = df_flights[['ARRIVAL_DELAY', 'ARRIVAL_TIME', 'SCHEDULED_ARRIVAL']]

df_flights_arrivals.sample(20)


,ARRIVAL_DELAY,ARRIVAL_TIME,SCHEDULED_ARRIVAL
4254412,1.0,1711.0,1710
161908,-21.0,1520.0,1541
4993973,-5.0,1945.0,1950
5574788,-14.0,1126.0,1140
3096864,-5.0,1614.0,1619
804254,-5.0,2245.0,2250
3813266,-4.0,1157.0,1201
3212176,-13.0,1540.0,1553
4143704,12.0,1648.0,1636
3485432,15.0,1530.0,1515


In [38]:
df_flights_arrivals = df_flights[['ARRIVAL_DELAY', 'ARRIVAL_TIME', 'SCHEDULED_ARRIVAL']]

df_flights_arrivals.loc[df_flights_arrivals['ARRIVAL_TIME'].isna()]


,ARRIVAL_DELAY,ARRIVAL_TIME,SCHEDULED_ARRIVAL
32,NaN,NaN,600
42,NaN,NaN,500
68,NaN,NaN,637
82,NaN,NaN,700
90,NaN,NaN,700
...,...,...,...
5818090,NaN,NaN,2300
5818157,NaN,NaN,2136
5818318,NaN,NaN,2256
5818777,NaN,NaN,50


In [ ]:
df_flights_arrivals = df_flights[['ARRIVAL_DELAY', 'ARRIVAL_TIME', 'SCHEDULED_ARRIVAL']]

df_flights_arrivals.loc[df_flights_arrivals['ARRIVAL_TIME'].isna()]

In [39]:
df_flights_NaNs_exploration = df_flights[['TAIL_NUMBER', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'WHEELS_ON', 'TAXI_IN', 'ARRIVAL_TIME', 'ARRIVAL_DELAY']]

### Set max rows and columns display DF

In [48]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df_NaNs = df_flights_NaNs_exploration.loc[df_flights_NaNs_exploration['TAIL_NUMBER'].isna()]

df_NaNs.isna().sum()

TAIL_NUMBER        14721
DEPARTURE_TIME     14721
DEPARTURE_DELAY    14721
TAXI_OUT           14721
WHEELS_OFF         14721
SCHEDULED_TIME         0
ELAPSED_TIME       14721
AIR_TIME           14721
WHEELS_ON          14721
TAXI_IN            14721
ARRIVAL_TIME       14721
ARRIVAL_DELAY      14721
dtype: int64

In [47]:
df_NaNs.shape

(14721, 12)

In [42]:
df_flights_dd.shape

(1018558, 6)

In [9]:
pd.set_option('display.max_rows', 500)

count = df_flights.value_counts('ORIGIN_AIRPORT')

count

ORIGIN_AIRPORT
ATL      346836
ORD      285884
DFW      239551
DEN      196055
LAX      194673
          ...  
13964         1
14025         1
14222         1
15497         1
12265         1
Name: count, Length: 930, dtype: int64

In [10]:
df_flights.value_counts('FLIGHT_NUMBER')

FLIGHT_NUMBER
469     3975
327     3554
326     3513
188     3386
403     3370
        ... 
6812       1
6828       1
6850       1
6881       1
9855       1
Name: count, Length: 6952, dtype: int64

Infos MIN/MAX des raisons pour les retards

In [11]:
print("min AIR_SYSTEM_DELAY:", df_flights['AIR_SYSTEM_DELAY'].min())
print("max AIR_SYSTEM_DELAY:", df_flights['AIR_SYSTEM_DELAY'].max())

print()

print("min SECURITY_DELAY:", df_flights['SECURITY_DELAY'].min())
print("max SECURITY_DELAY:", df_flights['SECURITY_DELAY'].max())

print()

print("min AIRLINE_DELAY:", df_flights['AIRLINE_DELAY'].min())
print("max AIRLINE_DELAY", df_flights['AIRLINE_DELAY'].max())


print()

print("min LATE_AIRCRAFT_DELAY:", df_flights['LATE_AIRCRAFT_DELAY'].min())
print("max LATE_AIRCRAFT_DELAY", df_flights['LATE_AIRCRAFT_DELAY'].max())

print()

print("min WEATHER_DELAY:", df_flights['WEATHER_DELAY'].min())
print("max WEATHER_DELAY", df_flights['WEATHER_DELAY'].max())



min AIR_SYSTEM_DELAY: 0.0
max AIR_SYSTEM_DELAY: 1134.0

min SECURITY_DELAY: 0.0
max SECURITY_DELAY: 573.0

min AIRLINE_DELAY: 0.0
max AIRLINE_DELAY 1971.0

min LATE_AIRCRAFT_DELAY: 0.0
max LATE_AIRCRAFT_DELAY 1331.0

min WEATHER_DELAY: 0.0
max WEATHER_DELAY 1211.0


In [12]:
df_flights.value_counts('CANCELLATION_REASON')

CANCELLATION_REASON
B    48851
A    25262
C    15749
D       22
Name: count, dtype: int64

### Initial EDA

In [13]:
df_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [14]:
df_flights.value_counts('TAIL_NUMBER')

TAIL_NUMBER
N480HA    3768
N484HA    3723
N488HA    3723
N493HA    3585
N478HA    3577
          ... 
N840MH       1
N860NW       1
N852NW       1
N180UA       1
N7LEAA       1
Name: count, Length: 4897, dtype: int64

In [15]:
df_flights.isna().sum()



YEAR                         0
MONTH                        0
DAY                          0
DAY_OF_WEEK                  0
AIRLINE                      0
FLIGHT_NUMBER                0
TAIL_NUMBER              14721
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME           86153
DEPARTURE_DELAY          86153
TAXI_OUT                 89047
WHEELS_OFF               89047
SCHEDULED_TIME               6
ELAPSED_TIME            105071
AIR_TIME                105071
DISTANCE                     0
WHEELS_ON                92513
TAXI_IN                  92513
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME             92513
ARRIVAL_DELAY           105071
DIVERTED                     0
CANCELLED                    0
CANCELLATION_REASON    5729195
AIR_SYSTEM_DELAY       4755640
SECURITY_DELAY         4755640
AIRLINE_DELAY          4755640
LATE_AIRCRAFT_DELAY    4755640
WEATHER_DELAY          4755640
dtype: int64

In [22]:
# Définir le nombre de lignes à afficher sur "None" pour afficher toutes les lignes
pd.set_option('display.max_rows', None)

# Si vous souhaitez également afficher toutes les colonnes
pd.set_option('display.max_columns', None)

In [28]:
df_flights.sample(10)


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
4715859,2015,10,21,3,EV,4862,N14902,12951,12266,1906,1851.0,-15.0,17.0,1908.0,69.0,70.0,45.0,201,1953.0,8.0,2015,2001.0,-14.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2434096,2015,6,3,3,AA,1640,N3CBAA,LGA,MIA,1950,1945.0,-5.0,15.0,2000.0,189.0,174.0,155.0,1096,2235.0,4.0,2259,2239.0,-20.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5138440,2015,11,17,2,EV,5107,N754EV,AGS,ATL,1852,1847.0,-5.0,38.0,1925.0,67.0,79.0,33.0,143,1958.0,8.0,1959,2006.0,7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4748989,2015,10,23,5,EV,4862,N14177,12951,12266,1906,2109.0,123.0,16.0,2125.0,69.0,65.0,42.0,201,2207.0,7.0,2015,2214.0,119.0,0,0,NaN,0.0,0.0,0.0,119.0,0.0
2116527,2015,5,15,5,WN,261,N7736A,SEA,MKE,650,651.0,1.0,16.0,707.0,225.0,228.0,207.0,1694,1234.0,5.0,1235,1239.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2047865,2015,5,11,1,AA,1219,N3MHAA,DCA,ORD,535,529.0,-6.0,10.0,539.0,132.0,108.0,88.0,612,607.0,10.0,647,617.0,-30.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1760692,2015,4,23,4,US,2144,N950UW,DCA,LGA,700,653.0,-7.0,30.0,723.0,79.0,84.0,46.0,214,809.0,8.0,819,817.0,-2.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4658527,2015,10,18,7,EV,4527,N14180,12339,11618,1058,1100.0,2.0,8.0,1108.0,121.0,119.0,95.0,645,1243.0,16.0,1259,1259.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3590551,2015,8,11,2,EV,4391,N11547,IAH,CRP,1735,1732.0,-3.0,77.0,1849.0,61.0,124.0,42.0,201,1931.0,5.0,1836,1936.0,60.0,0,0,NaN,60.0,0.0,0.0,0.0,0.0
289729,2015,1,20,2,US,2101,N824AW,BOS,DCA,600,556.0,-4.0,14.0,610.0,101.0,99.0,79.0,399,729.0,6.0,741,735.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:

df_flights.columns

df_short = df_flights[['TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']].head()

df_short.to_csv('df_short.csv', index = False)

# DF Planes

In [15]:
df_planes = pd.read_csv("/Users/kimberlyblack/Documents/POLE EMPLOI/DATA ANALYST/WCS/Projet 3/planes.csv")

#previsualisation
df_planes.head()

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2.0,182.0,NaN,Turbo-fan
1,N103SY,NaN,NaN,EMBRAER S A,ERJ 170-200 LR,NaN,NaN,NaN,NaN
2,N103SY,NaN,NaN,DEHAVILLAND,DHC-3,NaN,NaN,NaN,NaN
3,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2.0,182.0,NaN,Turbo-fan
4,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2.0,182.0,NaN,Turbo-fan


In [16]:
df_planes.columns

Index(['tailnum', 'year', 'type', 'manufacturer', 'model', 'engines', 'seats',
       'speed', 'engine'],
      dtype='object')

In [59]:
df_planes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   tailnum       4915 non-null   object 
 1   year          3493 non-null   float64
 2   type          3566 non-null   object 
 3   manufacturer  4915 non-null   object 
 4   model         4915 non-null   object 
 5   engines       3566 non-null   float64
 6   seats         3566 non-null   float64
 7   speed         24 non-null     float64
 8   engine        3566 non-null   object 
dtypes: float64(4), object(5)
memory usage: 345.7+ KB


In [63]:
df_planes.value_counts('manufacturer').sort_index()

manufacturer
3D ROBOTICS                          1
AERO COMMANDER                       2
AEROSPATIALE                         2
AGUSTA SPA                           2
AGUSTA SPA                           1
AIR TRACTOR INC                      1
AIRBUS                             346
AIRBUS                             211
AIRBUS INDUSTRIE                   430
AIRBUS INDUSTRIE                    42
AMERICAN                             1
AMERICAN AIRCRAFT INC                2
AMERICAN AIRCRAFT INC                1
ARNOLD ALAN WW                       1
AUTHIER ANTHONY P                    1
AVIAT AIRCRAFT INC                   1
AVIAT AIRCRAFT INC                   1
AVIONS MARCEL DASSAULT               1
BARKER JACK L                        1
BEECH                                2
BEECH                               35
BEER JOHN EDWARD                     1
BELL                                 5
BELL                                 1
BELL HELICOPTER TEXTRON CANADA       1
BENHAM JOHN 

In [65]:
df_planes['manufacturer'] = df_planes['manufacturer'].replace(r'\s{2,}', '', regex=True)
df_planes['manufacturer'] = df_planes['manufacturer'].replace(r'(INC|LTD|CORP|CO|INDUSTRIE)\s*$', '', regex=True)
df_planes['manufacturer'] = df_planes['manufacturer'].replace(r'\s\w$', '', regex=True)
df_planes['manufacturer'] = df_planes['manufacturer'].replace(r'\s$', '', regex=True)

In [66]:
df_planes.value_counts('manufacturer').sort_index()

manufacturer
3D ROBOTICS                          1
AERO COMMANDER                       2
AEROSPATIALE                         2
AGUSTA SPA                           3
AIR TRACTOR                          1
AIRBUS                            1029
AMERICAN                             1
AMERICAN AIRCRAFT                    3
ARNOLD ALAN WW                       1
AUTHIER ANTHONY                      1
AVIAT AIRCRAFT                       2
AVIONS MARCEL DASSAULT               1
BARKER JACK                          1
BEECH                               37
BEER JOHN EDWARD                     1
BELL                                 6
BELL HELICOPTER TEXTRON CANADA       1
BENHAM JOHN                          1
BOEING                            2267
BOMBARDIER                         682
BRYSON JEFF                          1
CAMERON MURDO                        1
CANADAIR                            19
CANNON B/LEWIS D                     1
CENTRAIR                             1
CESSNA      

# DF Cleaned_Aircraft_Data

In [2]:
df_cleaned_aircraft_data = pd.read_csv("/Users/kimberlyblack/Documents/GITHUB-projects/WCS-Project-4/WCS-Project_Late-Aircraft-Prediction-System_Dashboard_Customer-Review-Sentiment-Analysis/Cleaned_Aircraft_Data.csv")

#previsualisation
df_cleaned_aircraft_data.head()

,tailnum,year,type,manufacturer,model,engine_count,engine_type
0,100,1940.0,4,BENE MARY D,7100510,17003.0,1
1,10000,NaN,4,9AT LLC,2130004,NaN,1
2,10001,1928.0,4,STOOS ROBERT A,9601202,67007.0,1
3,10004,NaN,4,ETOS AIR LLC,2072738,NaN,2
4,10006,1955.0,4,COUTCHES ROBERT HERCULES DBA,1152020,17026.0,1


In [3]:
df_cleaned_aircraft_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299795 entries, 0 to 299794
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   tailnum       299795 non-null  object 
 1   year          242985 non-null  float64
 2   type          299795 non-null  object 
 3   manufacturer  299442 non-null  object 
 4   model         299795 non-null  object 
 5   engine_count  271894 non-null  float64
 6   engine_type   299795 non-null  int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 16.0+ MB


In [11]:
df_cleaned_aircraft_data.isnull().sum()

tailnum             0
year            56810
type                0
manufacturer      353
model               0
engine_count    27901
engine_type         0
dtype: int64

In [12]:
#check for unique values in engine_count column

print(df_cleaned_aircraft_data["engine_count"].unique())


[17003.    nan 67007. ... 41888. 41975. 41599.]


In [ ]:
#check for unique values in engine_type column

print(df_cleaned_aircraft_data["engine_type"].unique())


[ 1  2  0  5  8  7  3  4 10 11  9  6]


## Information about "engine_type" code

In [43]:
# Create dictionary with Aircraft ENGINE TYPE code

# Create a dictionary with column names as keys and lists as values
data = {
    0: "None",
    1: "Reciprocating",
    2: "Turbo-prop",
    3: "Turbo-shaft",
    4: "Turbo-jet",
    5: "Turbo-fan",
    6: "Ramjet",  
    7: "2 Cycle",
    8: "4 Cycle",
    9: "Unknown",
    10: "Electric",
    11: "Rotary"

}

# Convert the dictionary into a Pandas DataFrame
df = pd.DataFrame(list(data.items()), columns=["Code", "Aircraft ENGINE Type"])

# Display the DataFrame
print(df)


    Code Aircraft ENGINE Type
0      0                 None
1      1        Reciprocating
2      2           Turbo-prop
3      3          Turbo-shaft
4      4            Turbo-jet
5      5            Turbo-fan
6      6               Ramjet
7      7              2 Cycle
8      8              4 Cycle
9      9              Unknown
10    10             Electric
11    11               Rotary


In [32]:
see = rows_df_cleaned_aircraft_with_nan_values['manufacturer'].value_counts()
see

manufacturer
PIXIS DRONES LLC                   646
REGISTRATION PENDING               328
CAPE MAY AERIAL ADVERTISING LLC    326
SALE REPORTED                      253
BANK OF UTAH TRUSTEE               240
                                  ... 
PUSSER JOSHUA D                      1
WHITE BETH A                         1
WXS FLYING LLC                       1
COLIN TONY RAY                       1
SCENIC AVIATION LLC                  1
Name: count, Length: 42980, dtype: int64

In [33]:
#check for unique values in type column

print(df_cleaned_aircraft_data["type"].unique())

['4' '6' '1' '5' '2' '7' '8' 'H' '9' '3' 'O']


## Information about "type" code

In [54]:
# Create dictionary with Aircraft TYPE code

# Create a dictionary with column names as keys and lists as values
data_aircraft_type = {
    1: "Glider",
    2: "Balloon",
    3: "limp/Dirigible",
    4: "Fixed wing single engines",
    5: "Fixed wing multi engine",
    6: "Rotorcraft",  
    7: "Weight-shift-control",
    8: "Powered Parachute",
    9: "Gyroplane",
    "H": "Hybrid Lift",
    "O": "Other"

}


# Convert the dictionary into a Pandas DataFrame
df_aircraft_type_code = pd.DataFrame(list(data_aircraft_type.items()), columns=["Code", "Aircraft Type"])

# Display the DataFrame
print(df_aircraft_type_code)

   Code              Aircraft Type
0     1                     Glider
1     2                    Balloon
2     3             limp/Dirigible
3     4  Fixed wing single engines
4     5    Fixed wing multi engine
5     6                 Rotorcraft
6     7       Weight-shift-control
7     8          Powered Parachute
8     9                  Gyroplane
9     H                Hybrid Lift
10    O                      Other


In [45]:
#check for unique values in n- column

print(df_cleaned_aircraft_data["type"].unique())

['4' '6' '1' '5' '2' '7' '8' 'H' '9' '3' 'O']


In [46]:
#percentage of NAN values per column

nan_percent_cleaned_aircraft_data = df_cleaned_aircraft_data.isna().mean()*100

nan_percent_cleaned_aircraft_data

tailnum          0.000000
year            18.949616
type             0.000000
manufacturer     0.117747
model            0.000000
engine_count     9.306693
engine_type      0.000000
dtype: float64

In [47]:
#display rows with NAN values

rows_df_cleaned_aircraft_with_nan_values = df_cleaned_aircraft_data[df_cleaned_aircraft_data.isna().any(axis=1)]
see = rows_df_cleaned_aircraft_with_nan_values['manufacturer'].value_counts()
see

manufacturer
PIXIS DRONES LLC                   646
REGISTRATION PENDING               328
CAPE MAY AERIAL ADVERTISING LLC    326
SALE REPORTED                      253
BANK OF UTAH TRUSTEE               240
                                  ... 
PUSSER JOSHUA D                      1
WHITE BETH A                         1
WXS FLYING LLC                       1
COLIN TONY RAY                       1
SCENIC AVIATION LLC                  1
Name: count, Length: 42980, dtype: int64

In [49]:
# Count with the number of occurrences of each unique value, sort results by alphabetically order

df_cleaned_aircraft_data.value_counts('manufacturer').sort_index()

manufacturer
#430B LLC                    1
$100 HAMBURGER LLC           1
.5 MOON LLC                  1
0 TO 100 AVIATION LLC        1
007 ARKANSAS AVIATION LLC    1
                            ..
ZYGOMALAS STEFANOS           1
ZYGOWIEC KRYSTIAN            1
ZYLSTRA CYCLE CO INC         1
ZYSKOWSKI MICHAEL K          1
ZZ TOP RENTALS INC           1
Name: count, Length: 195727, dtype: int64

In [48]:
# sort by count of manufacturer column values

df_cleaned_aircraft_data.value_counts('manufacturer').sort_values(ascending=False)


manufacturer
BANK OF UTAH TRUSTEE                   1954
REGISTRATION PENDING                   1428
TVPX AIRCRAFT SOLUTIONS INC TRUSTEE    1397
UNITED AIRLINES INC                    1114
DELTA AIR LINES INC                    1068
                                       ... 
ROUTH LUCAS R                             1
ROUTMAN LEONE AIRCRAFT LLC                1
ROUTSON DAVID P                           1
ROUTT THOMAS                              1
ZZ TOP RENTALS INC                        1
Name: count, Length: 195727, dtype: int64

In [ ]:
# Check if 'manufacturer' contains "Boeing"
boeing_filter = df_cleaned_aircraft_data["manufacturer"].str.contains("Boeing", case=False, na=False)

# Display matching rows
print(df_cleaned_aircraft_data[boeing_filter])


       tailnum    year type                             manufacturer    model  \
4313      109X  1954.0    4              BOEING LOGISTICS SPARES INC  1901204   
23367    157AE  2011.0    6                           BOEING COMPANY  1145004   
28669    17359     NaN    5                                BOEING CO  13845FZ   
28679    17361  2024.0    5                                BOEING CO  13845FZ   
28680    17362     NaN    5                                BOEING CO  13845FZ   
...        ...     ...  ...                                      ...      ...   
285607   9537Q  1998.0    4             BOEING EMPOLYEES FLYING ASSN  2072401   
285931    954B  2023.0    4      BOEING EMPLOYEES FLYING ASSOCIATION  069002L   
287327   959AM     NaN    5                                BOEING CO  138470A   
288700   963DN  1999.0    5                           BOEING COMPANY  13848ME   
293332    97PD  2000.0    4  BOEING EMPLOYEES FLYING ASSOCIATION INC  2072439   

        engine_count  engin

## We need to determine what to do about the manufacturers that are part of the same global entity like Boeing.

In [ ]:
# Create dictionary with Aircraft ENGINE TYPE code

# Create a dictionary with column names as keys and lists as values
data_engine_type = {
    0: "None",
    1: "Reciprocating",
    2: "Turbo-prop",
    3: "Turbo-shaft",
    4: "Turbo-jet",
    5: "Turbo-fan",
    6: "Ramjet",  
    7: "2 Cycle",
    8: "4 Cycle",
    9: "Unknown",
    10: "Electric",
    11: "Rotary"


}


# Convert the dictionary into a Pandas DataFrame
df_engine_type_code = pd.DataFrame(data_engine_type)

# Display the DataFrame
print(df_engine_type_code)


ValueError: If using all scalar values, you must pass an index

# DF Airline Review

In [4]:
df_review = pd.read_csv("/Users/kimberlyblack/Documents/POLE EMPLOI/DATA ANALYST/WCS/Projet 3/Airline_review.csv")

#previsualisation
df_review.head()



,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,NaN,Solo Leisure,Economy Class,Moroni to Moheli,November 2019,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,E120,Solo Leisure,Economy Class,Moroni to Anjouan,June 2019,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,Embraer E120,Solo Leisure,Economy Class,Anjouan to Dzaoudzi,June 2019,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no
3,3,Adria Airways,1,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,NaN,Solo Leisure,Economy Class,Frankfurt to Pristina,September 2019,1.0,1.0,NaN,1.0,NaN,NaN,1.0,no
4,4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,Couple Leisure,Economy Class,Sofia to Amsterdam via Ljubljana,September 2019,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no


In [20]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23171 entries, 0 to 23170
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              23171 non-null  int64  
 1   Airline Name            23171 non-null  object 
 2   Overall_Rating          23171 non-null  object 
 3   Review_Title            23171 non-null  object 
 4   Review Date             23171 non-null  object 
 5   Verified                23171 non-null  bool   
 6   Review                  23171 non-null  object 
 7   Aircraft                7129 non-null   object 
 8   Type Of Traveller       19433 non-null  object 
 9   Seat Type               22075 non-null  object 
 10  Route                   19343 non-null  object 
 11  Date Flown              19417 non-null  object 
 12  Seat Comfort            19016 non-null  float64
 13  Cabin Staff Service     18911 non-null  float64
 14  Food & Beverages        14500 non-null

In [5]:
df_review.columns

Index(['Unnamed: 0', 'Airline Name', 'Overall_Rating', 'Review_Title',
       'Review Date', 'Verified', 'Review', 'Aircraft', 'Type Of Traveller',
       'Seat Type', 'Route', 'Date Flown', 'Seat Comfort',
       'Cabin Staff Service', 'Food & Beverages', 'Ground Service',
       'Inflight Entertainment', 'Wifi & Connectivity', 'Value For Money',
       'Recommended'],
      dtype='object')

In [ ]:
df_review

# EDA of the cleaned dataset

In [ ]:
df_flights_clean = pd.read_csv("/Users/kimberlyblack/Documents/POLE EMPLOI/DATA ANALYST/WCS/Projet 3/df_flights_clean.csv")

#previsualisation
df_flights_clean.sample(20)


# DF Airline Delay Cause

In [2]:
df_airline_delay_cause = pd.read_csv("/Users/kimberlyblack/Documents/POLE EMPLOI/DATA ANALYST/WCS/Projet 3/Airline_Delay_Cause.csv")

#previsualisation
df_airline_delay_cause.sample(20)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
355,2024,6,MQ,Envoy Air,SAV,"Savannah, GA: Savannah/Hilton Head International",12.0,4.0,0.54,0.00,...,0.00,3.44,0.0,0.0,405.0,71.0,0.0,1.0,0.0,333.0
1056,2024,6,WN,Southwest Airlines,MHT,"Manchester, NH: Manchester Boston Regional",222.0,62.0,32.45,0.97,...,0.00,21.02,0.0,0.0,3077.0,1524.0,138.0,201.0,0.0,1214.0
933,2024,6,UA,United Air Lines Network,LAS,"Las Vegas, NV: Harry Reid International",1063.0,238.0,72.25,9.82,...,0.00,92.06,6.0,1.0,15845.0,4394.0,1456.0,2575.0,0.0,7420.0
614,2024,6,OO,SkyWest Airlines Inc.,FOD,"Fort Dodge, IA: Fort Dodge Regional",51.0,16.0,5.64,0.00,...,0.00,9.48,0.0,0.0,1326.0,418.0,0.0,36.0,0.0,872.0
1910,2024,6,G7,GoJet Airlines LLC d/b/a United Express,GSP,"Greer, SC: Greenville-Spartanburg International",8.0,2.0,1.00,0.00,...,0.00,0.00,0.0,0.0,56.0,27.0,0.0,29.0,0.0,0.0
1673,2024,6,DL,Delta Air Lines Network,SRQ,"Sarasota/Bradenton, FL: Sarasota/Bradenton Int...",211.0,34.0,19.60,2.62,...,0.00,4.73,0.0,2.0,1982.0,972.0,193.0,373.0,0.0,444.0
1169,2024,6,9E,Endeavor Air Inc.,MLI,"Moline, IL: Quad Cities International",60.0,10.0,4.64,0.11,...,0.00,2.57,0.0,0.0,466.0,193.0,10.0,158.0,0.0,105.0
516,2024,6,OH,PSA Airlines Inc.,SRQ,"Sarasota/Bradenton, FL: Sarasota/Bradenton Int...",49.0,4.0,0.25,0.00,...,0.00,1.75,0.0,0.0,233.0,20.0,0.0,39.0,0.0,174.0
648,2024,6,OO,SkyWest Airlines Inc.,JAX,"Jacksonville, FL: Jacksonville International",32.0,4.0,2.00,1.82,...,0.00,0.00,0.0,0.0,304.0,47.0,216.0,41.0,0.0,0.0
130,2024,6,YX,Republic Airline,RIC,"Richmond, VA: Richmond International",223.0,28.0,10.50,0.88,...,0.00,7.37,9.0,1.0,1989.0,975.0,89.0,467.0,0.0,458.0


In [3]:
df_airline_delay_cause.columns

Index(['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct',
       'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
       'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay'],
      dtype='object')

# DF Data-FYYTg 

In [12]:
df_data_fyytg= pd.read_csv("/Users/kimberlyblack/Documents/POLE EMPLOI/DATA ANALYST/WCS/Projet 3/data-FYYTg.csv")

#previsualisation
df_data_fyytg.head()


,X.1,Mostly read,Same,Mostly watch,Don't know
0,55+,67,14,11,8
1,35+,65,16,12,8
2,25-34,60,18,14,8
3,18-24,55,19,17,9
